# Reto 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
data_pulsar = pd.read_csv('./data/HTRU_2.csv', header=None)

In [ ]:
x= data_pulsar.iloc[:,0:8]
y= data_pulsar.iloc[:,8:9]

In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, train_size = 10000, random_state = 0)

In [ ]:
newData= pd.concat([xTrain,yTrain], axis = 1)

In [ ]:
dataTrain1= newData.sample(100)
xTrain1= dataTrain1.iloc[:,0:8]
yTrain1= dataTrain1.iloc[:,8]
dataTrain2= newData.sample(500)
xTrain2= dataTrain2.iloc[:,0:8]
yTrain2= dataTrain2.iloc[:,8]
dataTrain3= newData.sample(1000)
xTrain3= dataTrain3.iloc[:,0:8]
yTrain3= dataTrain3.iloc[:,8]
dataTrain4= newData.sample(5000)
xTrain4= dataTrain4.iloc[:,0:8]
yTrain4= dataTrain4.iloc[:,8]

In [ ]:
logisticRegr1 = LogisticRegression(solver='liblinear')
logisticRegr1.fit(xTrain1, yTrain1)
logisticRegr2 = LogisticRegression(solver='liblinear')
logisticRegr2.fit(xTrain2, yTrain2)
logisticRegr3 = LogisticRegression(solver='liblinear')
logisticRegr3.fit(xTrain3, yTrain3)
logisticRegr4 = LogisticRegression(solver='liblinear')
logisticRegr4.fit(xTrain4, yTrain4)

In [ ]:
score1 = logisticRegr1.score(xTest, yTest)
print(score1)

In [ ]:
score2 = logisticRegr2.score(xTest, yTest)
print(score2)

In [ ]:
score3 = logisticRegr3.score(xTest, yTest)
print(score3)

In [ ]:
score4 = logisticRegr4.score(xTest, yTest)
print(score4)